In [157]:
import pandas as pd
import numpy as np
# !pip install folium
import folium
from folium.plugins import MarkerCluster

In [133]:
import torch
import torch.nn.functional as F

from transformers import BertForSequenceClassification, BertConfig
from transformers import BertTokenizer

In [134]:
warung_pintar_data = pd.read_csv('wp_review_gmaps_191175109.csv')
warung_pintar_data.head()

,id,Business_Name,Address,Rating,Total_Reviews,Lat,Lng,URL,User_Name,User_URL,...,User_Avatar,User_Reviews,Publish_date,Stars,Stars_External,Source,Review,Review_Photos,Owner_Response,Unnamed: 20
0,31,Warung Pintar,"Kendal, Sragi, Songgon, Banyuwangi Regency, Ea...",5.0,(1),-8.258615,114.177442,https://www.google.com/maps/place/Warung+Pinta...,Noer Hoeda,https://www.google.com/maps/contrib/1054841379...,...,https://lh6.googleusercontent.com/-uM71JNMnj5g...,1 review,9 months ago,5 stars,NaN,NaN,"(Translated by Google) Coffee, snack, snack .....",https://lh5.googleusercontent.com/p/AF1QipPWqA...,NaN,NaN
1,30,Warung pintar,"Jl. H Ikhsan, Sukomukti, Kesilir, Siliragung, ...",1.0,(1),-8.524533,114.110316,https://www.google.com/maps/place/Warung+pinta...,Master Geblek,https://www.google.com/maps/contrib/1180506209...,...,https://lh6.googleusercontent.com/-dEjDjqYQOOI...,NaN,11 months ago,1 star,NaN,NaN,NaN,NaN,NaN,NaN
2,29,WARUNG PINTAR SUMBERWARU1,"sumberwaru, Sumberwaru, Tamanagung, Cluring, K...",5.0,(2),-8.414871,114.209468,https://www.google.com/maps/place/WARUNG+PINTA...,BENY GUNTORO,https://www.google.com/maps/contrib/1006872349...,...,https://lh4.googleusercontent.com/-W71QajaHG-U...,NaN,7 months ago,5 stars,NaN,NaN,NaN,NaN,NaN,NaN
3,28,WARUNG PINTAR SUMBERWARU1,"sumberwaru, Sumberwaru, Tamanagung, Cluring, K...",5.0,(2),-8.414871,114.209468,https://www.google.com/maps/place/WARUNG+PINTA...,Adhiatma D Saputra,https://www.google.com/maps/contrib/1092482470...,...,https://lh6.googleusercontent.com/-ljn53OR3co4...,2 reviews,2 months ago,5 stars,NaN,NaN,"(Translated by Google) The trader is funny,\r\...",NaN,NaN,NaN
4,27,Warung Pintar Rakyat,"Sentong, Komis Wetan, Wonosobo, Kec. Srono, Ka...",5.0,(2),-8.390776,114.289773,https://www.google.com/maps/place/Warung+Pinta...,rudi golkar,https://www.google.com/maps/contrib/1097027652...,...,https://lh3.googleusercontent.com/-LtZemhZ92no...,2 reviews,7 months ago,5 stars,NaN,NaN,"(Translated by Google) Comfortable, smart, and...",https://lh5.googleusercontent.com/p/AF1QipN0tl...,NaN,NaN


In [135]:
warung_pintar_data.columns

Index(['id', 'Business_Name', 'Address', 'Rating', 'Total_Reviews', 'Lat',
       'Lng', 'URL', 'User_Name', 'User_URL', 'User_Title', 'User_Avatar',
       'User_Reviews', 'Publish_date', 'Stars', 'Stars_External', 'Source',
       'Review', 'Review_Photos', 'Owner_Response', 'Unnamed: 20'],
      dtype='object')

In [136]:
wp_df = warung_pintar_data[['Business_Name', 
                            'Address', 
                            'Rating',
                            'Lat', 'Lng',
                            'Stars',
                            'Review']]
wp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Business_Name  31 non-null     object 
 1   Address        31 non-null     object 
 2   Rating         31 non-null     float64
 3   Lat            31 non-null     float64
 4   Lng            31 non-null     float64
 5   Stars          31 non-null     object 
 6   Review         19 non-null     object 
dtypes: float64(3), object(4)
memory usage: 1.8+ KB


In [137]:
wp_df.head()

,Business_Name,Address,Rating,Lat,Lng,Stars,Review
0,Warung Pintar,"Kendal, Sragi, Songgon, Banyuwangi Regency, Ea...",5.0,-8.258615,114.177442,5 stars,"(Translated by Google) Coffee, snack, snack ....."
1,Warung pintar,"Jl. H Ikhsan, Sukomukti, Kesilir, Siliragung, ...",1.0,-8.524533,114.110316,1 star,NaN
2,WARUNG PINTAR SUMBERWARU1,"sumberwaru, Sumberwaru, Tamanagung, Cluring, K...",5.0,-8.414871,114.209468,5 stars,NaN
3,WARUNG PINTAR SUMBERWARU1,"sumberwaru, Sumberwaru, Tamanagung, Cluring, K...",5.0,-8.414871,114.209468,5 stars,"(Translated by Google) The trader is funny,\r\..."
4,Warung Pintar Rakyat,"Sentong, Komis Wetan, Wonosobo, Kec. Srono, Ka...",5.0,-8.390776,114.289773,5 stars,"(Translated by Google) Comfortable, smart, and..."


In [138]:
wp_df['Review'].fillna('None', inplace=True)

C:\Users\Ahmad Shohibus S\AppData\Roaming\Python\Python37\site-packages\pandas\core\series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [139]:
wp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Business_Name  31 non-null     object 
 1   Address        31 non-null     object 
 2   Rating         31 non-null     float64
 3   Lat            31 non-null     float64
 4   Lng            31 non-null     float64
 5   Stars          31 non-null     object 
 6   Review         31 non-null     object 
dtypes: float64(3), object(4)
memory usage: 1.8+ KB


In [144]:
def extractAndCleanReview(review):
    if review == '':
        return ''
    else:
        import re
        review = re.sub('\W+',' ', review)
        review = review.strip()
        review = review.replace('\r', '').replace('\n', '')
        if "Original" in review:
            review = review.split("Original")[1]
        return review

In [146]:
wp_df['textReview'] = wp_df['Review'].apply(extractAndCleanReview)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [147]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained("indobenchmark/indobert-base-p1", num_labels=3)
model = BertForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p1", config=config)
model.load_state_dict(torch.load("model.bin"))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [154]:
i2w = {0: 'positive', 1: 'neutral', 2: 'negative'}
w2i = {'positive': 0, 'neutral': 1, 'negative': 2}

In [150]:
def classifySentiment(text):
    subwords = tokenizer.encode(text)
    subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

    logits = model(subwords)[0]
    label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
    return i2w[label]

In [152]:
wp_df[['label']] = wp_df['textReview'].apply(classifySentiment)

C:\Users\Ahmad Shohibus S\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Ahmad Shohibus S\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [153]:
wp_df.head()

,Business_Name,Address,Rating,Lat,Lng,Stars,Review,textReview,label
0,Warung Pintar,"Kendal, Sragi, Songgon, Banyuwangi Regency, Ea...",5.0,-8.258615,114.177442,5 stars,"(Translated by Google) Coffee, snack, snack .....",Ngopi ngemi ngemil dll full music dan free wifi,positive
1,Warung pintar,"Jl. H Ikhsan, Sukomukti, Kesilir, Siliragung, ...",1.0,-8.524533,114.110316,1 star,None,None,negative
2,WARUNG PINTAR SUMBERWARU1,"sumberwaru, Sumberwaru, Tamanagung, Cluring, K...",5.0,-8.414871,114.209468,5 stars,None,None,negative
3,WARUNG PINTAR SUMBERWARU1,"sumberwaru, Sumberwaru, Tamanagung, Cluring, K...",5.0,-8.414871,114.209468,5 stars,"(Translated by Google) The trader is funny,\r\...",Pedagangnya lucu Konsumennya lebih lucu,positive
4,Warung Pintar Rakyat,"Sentong, Komis Wetan, Wonosobo, Kec. Srono, Ka...",5.0,-8.390776,114.289773,5 stars,"(Translated by Google) Comfortable, smart, and...",Nyaman mencerdaskan dan mengasyikkan,positive


In [159]:
wp_df['color'] = wp_df['label'].map({'positive' : 'lightblue', 'neutral': 'lightgreen', 'negative': 'lightred'})

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [168]:
wp_df['text'] = 'Nama WP : \n' + wp_df['Business_Name'] + ', Bintang : \n' + wp_df['Stars'] + ', Review : \n' + wp_df['textReview']

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [169]:
locations = wp_df[['Lat', 'Lng']]
locationslist = locations.values.tolist()

arrtext = wp_df.text.to_numpy()
arrcolor = wp_df.color.to_numpy()

makercluster = MarkerCluster()
mapfolium = folium.Map(location=[-8.2169235,114.3311312], zoom_start=12)
makercluster = makercluster.add_to(mapfolium)

for point in range(0, len(locationslist)):
    folium.Marker(locationslist[point], popup = arrtext[point], icon = folium.Icon(color=arrcolor[point], icon_color='white',
                     icon='male', angle=0, prefix='fa')).add_to(makercluster)

In [170]:
mapfolium

In [171]:
mapfolium.save('foliummap.html')